# View Tendências Sensores

Tendências de leituras de sensores por equipamento e tipo.

In [ ]:
# Configuração (lê parâmetros do job)
try:
    CATALOG = dbutils.widgets.get('catalog')
except:
    CATALOG = 'manufatura_lakehouse'

try:
    SCHEMA_SILVER = dbutils.widgets.get('schema_silver')
except:
    SCHEMA_SILVER = 'silver'

try:
    SCHEMA_GOLD = dbutils.widgets.get('schema_gold')
except:
    SCHEMA_GOLD = 'gold'

def fqtn(schema, table):
    if CATALOG and CATALOG.lower() != 'none':
        return f"`{CATALOG}`.`{schema}`.`{table}`"
    else:
        return f"`{schema}`.`{table}`"

# Criar schema se necessário
if CATALOG and CATALOG.lower() != 'none':
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS `{CATALOG}`.`{SCHEMA_GOLD}`")
    spark.sql(f"USE CATALOG `{CATALOG}`")
else:
    spark.sql(f"CREATE DATABASE IF NOT EXISTS `{SCHEMA_GOLD}`")


In [ ]:
spark.sql(f"""
CREATE OR REPLACE VIEW {fqtn(SCHEMA_GOLD, 'vw_tendencias_sensores')} AS
SELECT 
    t.full_date,
    e.equipment_name,
    i.sensor_type,
    avg(i.avg_value) as avg_reading,
    min(i.min_value) as min_reading,
    max(i.max_value) as max_reading,
    sum(i.reading_count) as total_readings
FROM {fqtn(SCHEMA_GOLD, 'fact_iot_agregado')} i
JOIN {fqtn(SCHEMA_GOLD, 'dim_tempo')} t ON i.date_key = t.date_key
JOIN {fqtn(SCHEMA_GOLD, 'dim_equipamento')} e ON i.equipment_id = e.equipment_id
GROUP BY 1, 2, 3
ORDER BY t.full_date DESC, e.equipment_name, i.sensor_type
""")